In [1]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import re

station_people = pd.read_csv('./data/subway.csv',encoding='EUC-KR')

#승차
in_subway= station_people.columns.tolist()[:3]
for a in station_people.columns.tolist():
    if a.find('승차')!= -1 :
        in_subway.append(a)

in_subway_df = station_people[in_subway]
station_group_in=in_subway_df.groupby(['호선명','지하철역']).sum()
station_group_in=station_group_in.drop(['사용월'],axis=1)

#하차
out_subway= station_people.columns.tolist()[:3]
for a in station_people.columns.tolist():
    if a.find('하차')!= -1 :
        out_subway.append(a)

out_subway_df = station_people[out_subway]
station_group_out=out_subway_df.groupby(['호선명','지하철역']).sum()
station_group_out=station_group_out.drop(['사용월'],axis=1)

#승차 엑셀 저장
station_group_in.style.background_gradient().to_excel('./result/승차_인원.xlsx', engine='openpyxl')

#하차 엑셀 저장
station_group_out.style.background_gradient().to_excel('./result/하차_인원.xlsx', engine='openpyxl')

In [2]:
address_station = pd.read_csv('./data/subway.csv',encoding='EUC-KR')

# 호선명과 지하철 역 정보만 추출
address_station=address_station.loc[address_station['사용월']==202304,:]
address_station=address_station[['호선명','지하철역']]

#이름에 '역' 넣기
for i in address_station.index:
    station=''.join(list(address_station.loc[i,['지하철역']]))
    if station[-1] != '역':
        address_station.loc[i,['지하철역']] = station+'역'

#괄호 안의 내용 삭제
def remove_brackets(text):
    return re.sub(r'\([^)]*\)', '', text)

address_station['지하철역'] = address_station['지하철역'].apply(remove_brackets)

address_station

,호선명,지하철역
0,1호선,동대문역
1,1호선,동묘앞역
2,1호선,서울역
3,1호선,시청역
4,1호선,신설동역
...,...,...
608,중앙선,원덕역
609,중앙선,중랑역
610,중앙선,지평역
611,중앙선,팔당역


In [3]:
subway_location = pd.read_csv('./data/subway_location.csv',encoding='EUC-KR')

#이름에 '역' 넣기
for i in subway_location.index:
    station=''.join(list(subway_location.loc[i,['역이름']]))
    if station[-1] != '역':
        subway_location.loc[i,['역이름']] = station+'역'
        
subway_location

,역이름,y,x
0,가락시장역,37.492522,127.118234
1,종로3가역,37.571607,126.991806
2,오금역,37.502162,127.128111
3,동대문역,37.571420,127.009745
4,동대문역사문화공원역,37.565138,127.007896
...,...,...,...
584,동백역,37.269043,127.152716
585,초당역,37.260752,127.159443
586,삼가역,37.242115,127.168075
587,시청·용인대역,37.239151,127.178406


In [4]:
lat = []  #위도
lng = []  #경도

places = list(address_station['지하철역'])

i=0
for place in places:   
    i = i + 1
    try:
        location = subway_location[subway_location['역이름'] == place]
        lat.append(location['y'].values[0])
        lng.append(location['x'].values[0])
        

    except:
        lat.append('0.0')
        lng.append('0.0')

address_station['위도'] = lat
address_station['경도'] = lng

# 동일한 역 처리
address_station.loc[(address_station['지하철역']=='양평역')&(address_station['호선명']=='5호선'),['위도']]= 37.526226
address_station.loc[(address_station['지하철역']=='양평역')&(address_station['호선명']=='5호선'),['경도']]= 126.8844202

address_station.loc[(address_station['지하철역']=='양평역')&(address_station['호선명']=='중앙선'),['위도']]= 37.4927357
address_station.loc[(address_station['지하철역']=='양평역')&(address_station['호선명']=='중앙선'),['경도']]= 127.4896358

drops = address_station[address_station['경도'] == 0.0].index.to_list()
for drop in drops:
    address_station.drop([drop], inplace=True)

drops = address_station[address_station['경도'] == '0.0'].index.to_list()
for drop in drops:
    address_station.drop([drop], inplace=True)

address_station
address_station.to_csv("./result/station_address.csv",index=False)

In [7]:
# 서울시 중심부의 위도, 경도
center = [37.541, 126.986]

m = folium.Map(location=center, zoom_start=10)

c_dic = {'1호선':'darkblue','2호선':'green','3호선':'orange','4호선':'lightblue','5호선':'darkpurple','6호선':'black','7호선':'darkgreen','8호선':'pink','9호선':'beige','중앙선':'blue',
        '9호선2~3단계':'beige','경강선':'cadetblue','경부선':'cadetblue','경원선':'cadetblue','경의선':'cadetblue','경인선':'cadetblue','경춘선':'blue','공항철도 1호선':'cadetblue','과천선':'cadetblue','분당선':'lightred'
         ,'수인선':'lightred','안산선':'red','우이신설선':'cadetblue','일산선':'red','장항선':'cadetblue', '신림선':'cadetblue'}

for i in address_station.index:
    folium.Marker(
        location = list(address_station.loc[i,['위도', '경도']]),
        popup = (address_station.loc[i, '호선명'], address_station.loc[i, '지하철역']),
        tooltip = (address_station.loc[i, '호선명'], address_station.loc[i, '지하철역']),
        icon = folium.Icon(color=c_dic[address_station.loc[i, '호선명']], icon='star')
        ).add_to(m)
m.save("./result/Marker.html")
m

In [6]:
# 서울시 중심부의 위도, 경도
center = [37.541, 126.986]

m = folium.Map(location=center, zoom_start=10)

c_dic = {'1호선':'darkblue','2호선':'green','3호선':'orange','4호선':'lightblue','5호선':'darkpurple','6호선':'black','7호선':'darkgreen','8호선':'pink','9호선':'beige','중앙선':'blue',
        '9호선2~3단계':'beige','경강선':'cadetblue','경부선':'cadetblue','경원선':'cadetblue','경의선':'cadetblue','경인선':'cadetblue','경춘선':'blue','공항철도 1호선':'cadetblue','과천선':'cadetblue','분당선':'lightred'
         ,'수인선':'lightred','안산선':'red','우이신설선':'cadetblue','일산선':'red','장항선':'cadetblue', '신림선':'cadetblue'}

for i in address_station.index:
    folium.Circle(
        location = list(address_station.loc[i,['위도', '경도']]),
        radius = 80,
        popup = (address_station.loc[i, '호선명'], address_station.loc[i, '지하철역']),
        tooltip = (address_station.loc[i, '호선명'], address_station.loc[i, '지하철역']),
        fill = c_dic[address_station.loc[i, '호선명']],
        fill_opacity = 100,
        color = c_dic[address_station.loc[i, '호선명']]
        ).add_to(m)

HeatMap(
    data = address_station[['위도', '경도']],
    radius = 25
    ).add_to(m)
m.save("./result/HeatMap.html")
m